<a href="https://colab.research.google.com/github/Kyrylo-Bakumenko/NFL_Gamba_And_Power_Rankings/blob/main/2021WeeklyPredictionsFromModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
#@title Imports and Settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import time
from tqdm import tqdm
from numba import jit, njit
from numba import prange
from scipy.stats import invgauss
%pylab inline

# mount google drive
from google.colab import drive
drive.mount('/content/drive/')

# quality of life
pd.set_option('display.float_format', lambda x: '%.5f' % x)

Populating the interactive namespace from numpy and matplotlib
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [54]:
#@title User made functions

# return probability left wins in accordance to elo system
@njit
def win_prob(player_elo, opponent_elo):
  return 1/(1+10**((-player_elo+opponent_elo)/400))

# return new elo for player
def elo_change(player_elo, opponent_elo, result, DELTA=1):
  player_win_prob = win_prob(player_elo, opponent_elo)
  return player_elo + (MAX_GAIN*DELTA) * (result-player_win_prob)

# generates list of season names
def get_season_names(start, end, step=1):
    seasons = ["Reference"]
    while start+step < end:
      start+=step
      seasons.append("'"+str(start)[-2:])
    seasons.append("'"+str(end)[-2:])
    return seasons

  ### Web scraping method

def scraped_season_list(START_YEAR, END_YEAR):
  seasons = []
  for year in range(START_YEAR, END_YEAR):
    try:
      df = pd.read_html(URL_BASE + str(year) + URL_END, header=0)[0]
      df.drop(columns=['Day', 'Week', 'Time', 'Unnamed: 5', 'Unnamed: 7', 'YdsW', 'TOW', 'YdsL', 'TOL'], inplace=True)
      df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
      df.dropna(inplace=True)
      df.set_index('Date', inplace=True)
      seasons.append(df)
    except:
      print(f'Failed at year: {year}')
      break
  return seasons

### Deprecated file method
def create_season_list_by_years(START_YEAR, END_YEAR):
  seasons = []
  for i in range(START_YEAR, END_YEAR):
    try:
      seasons.append(pd.read_excel("/content/NFL_Season_Data.xlsx", sheet_name=str(i), header=0, usecols="B:F", index_col=0))
    except:
      break
  return seasons
###

def list_rename_dropna(seasons):
  for cur_season in seasons:
    # rename because we will check for draws
    cur_season.rename(columns={"Winner/tie": "Left", "Loser/tie": "Right"}, inplace=True)
    # remove nan's (artifacts from website)
    cur_season.dropna(inplace=True)
    # bridge name changes by using consistent abbreviations
    cur_season["Left"]=cur_season["Left"].map(name_to_abbr).values
    cur_season["Right"]=cur_season["Right"].map(name_to_abbr).values
  return seasons

def bet_prop_simple(bookies_odds, our_odds):
  return 2*(invgauss.cdf(min(our_odds/bookies_odds, 3), mu=30)-0.5)

def add_simulated_odds(Elo, game_bets):
  game_bets["Our Odds"] = 0
  for i in range(1, len(game_bets), 2): # every game in game_bets
    # find team in Elo, add our simulated odds
    #   0      1        2         3
    # Team    ML  Bookies Odds Our Odds
    team_a = game_bets.iloc[i-1, 0]
    team_b = game_bets.iloc[i, 0]
    # odds a wins according to elo model
    a_win_prob = win_prob(Elo[team_a], Elo[team_b])
    # odds b wins according to elo model
    b_win_prob = 1-a_win_prob
    # add to table
    game_bets.iloc[i-1, 3] = a_win_prob
    game_bets.iloc[i, 3] = b_win_prob


def add_recommended_wagers(game_bets):
  bet_size = 1
  method_profit = 0
  game_bets['Wager'] = 0
  # ...   0      1         2          3         4
  # ... Team    ML   Bookies Odds  Our Odds   Wager
  for game_num in range(len(game_bets)):
    BO = game_bets.iloc[game_num, 2] # money line implied odds
    OO = game_bets.iloc[game_num, 3] # our odds
    if OO > BO: # we have calculated an edge
      bet = bet_size*bet_prop_simple(BO, OO) # bet amount
      game_bets.iloc[game_num, 4] = bet # add bet to table

In [55]:
#@title Import and Manage target season odds and games
# import moneyline betting data for 2020-21 season
url = "/content/drive/My Drive/NFL_Analysis_Data/NFL_Odds_2021-22.xlsx"

game_bets = pd.read_excel(url, sheet_name="Week1", header=0,
                          usecols="B, C")#, index_col=0)

# convert ML into effective odds bookies are estimating
game_bets["Bookies Odds"] = 0

game_bets.head(50)

for row in range(game_bets["ML"].size):
  ML = game_bets["ML"][row]
  if ML > 0: # positive - underdog
    game_bets.iloc[row, 2] = 100/(ML+100)
  else: # negative - favorite
    game_bets.iloc[row, 2] = abs(ML)/(abs(ML)+100)

game_bets.head(5)

,Team,ML,Bookies Odds
0,DAL,325,0.23529
1,TB,-425,0.80952
2,JAX,-180,0.64286
3,HOU,150,0.40000
4,PIT,235,0.29851


In [56]:
Elo = pickle.load(open("elo.p", "rb" ))
add_simulated_odds(Elo, game_bets)
add_recommended_wagers(game_bets)